In [ ]:
!pip install fbprophet

In [ ]:
import datetime
import urllib
import requests
import pandas
import fbprophet

In [ ]:
PROMETHEUS_HOST = 'http://thanos-querier.istio-system.svc.cluster.local.:9090'
query = 'sum(rate(envoy_requests_total[1m])) by (host, status)'
start = str(int(datetime.datetime(2020, 4, 1, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(hours=9))).timestamp()))
end = str(int(datetime.datetime(2020, 5, 1, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(hours=9))).timestamp()))
step = '300s'
freq = '5min'
periods = 12
changepoint_range = 0.8
changepoint_prior_scale = 0.05
n_changepoints = 25
seasonality_mode = 'additive'
seasonality_prior_scale = 10.0
cross_validation_initial = '20day'
cross_validation_period = '1day'
cross_validation_horizon = '5day'
cross_validation_metric = 'mse'

In [ ]:
result = requests.get(PROMETHEUS_HOST + '/api/v1/query_range?query=' + urllib.parse.quote(query) + "&start=" + start + "&end=" + end + "&step=" + step).json()['data']['result']

In [ ]:
def p(r):
    print(r['metric'])
    if len(r['values']) < 2:
        return
    df = pandas.DataFrame([[datetime.datetime.fromtimestamp(value[0]), value[1]] for value in r['values']])
    df.columns = ['ds', 'y']
    p = fbprophet.Prophet(changepoint_range=changepoint_range,changepoint_prior_scale=changepoint_prior_scale,n_changepoints=n_changepoints,seasonality_mode=seasonality_mode,seasonality_prior_scale=seasonality_prior_scale)
    p.fit(df)
    forecast = p.predict(p.make_future_dataframe(periods=periods, freq=freq))
    p.plot(forecast)
    p.plot_components(forecast)
    cv = fbprophet.diagnostics.cross_validation(p, initial=cross_validation_initial, period=cross_validation_period, horizon=cross_validation_horizon)
    fbprophet.plot.plot_cross_validation_metric(cv, metric=cross_validation_metric)

In [ ]:
p(result[0])